# Other Resources
Here are some additional tutorials and resources that may be useful as points of reference:

1. [Data Carpentry's Geospatial Workshop](https://datacarpentry.org/geospatial-workshop/)

## Housekeeping and installations
First, some housekeeping. On linux/mac machines, dependencies are easy to build, but on Windows machines, things get weird quickly.

For this notebook, we'll make sure it runs fine in the binder environment (yay!), but we want you to have some notes so that you don't throw your computer out the window the first time you try to apply what you've learned outside of the notebook.

Anaconda/conda environments are often a solution, but I've seen `geopandas` break conda environments (entirely) as often as I've successfully installed it, so I recommend a different approach. Use your preferred environment manager, and install the wheels directly from Christopher Gohlke, who builds current wheels for current Python versions of common spatial software, after which it's safe to `pip install` geopandas.

### Linux/Mac
Here's what that looks like. On Linux/Mac, `python -m pip install geopandas` should get you the whole stack, though you may need to install some system packages for gdal (gdal-bin, gdal-dev) first.

If you want to use geopandas on one of these environments though, you should be safe to take a look at both the:
1. [`gdal` installation instructions](https://pypi.org/project/GDAL/) and
2. the [`geopandas` installation instructions](https://geopandas.org/en/stable/getting_started/install.html)

### On Windows



All of that is subject to change - these aren't definitive, but are meant to give you a spot to start after this workshop. The spatial python ecosystem is dispersed, so you're not stuck trying to work with geopandas, but it's powerful and may work well with many workflows RSEs already use.

https://github.com/nickrsan/spatial_resources/blob/main/installing_spatial_python_windows.md